<div style="font-size:18px;"> <span style="color:purple">
Data Science 100 Final Project :D

Introduction:
There is a research group at UBC that is collecting data on how people play video games. They have collected data on players through a Minecreaft Server. They have presented us with many broad questions that we need to explore. The broad question that has been chosen for the project today is Which "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts. From this we have formulated a predictuve question, the question we are trying to answer is Can we predict the number of sessions players partake in a Minecraft server based on their age?

This project uses two datasets that were taken from the minecraft server, the two CSV files are:
players.csv (This dataset has information on the players)
sessions.csv(This dataset has specific information on the players sessions)

These two datasets were cleaned and joined together by hashedEmail to create a dataset that we used to answer our question. 
|**Variable Name**  | **Type** |**Description**                     |
|-------------------|----------|------------------------------------|
|hashedEmail        |Character |An identifier for each player       |
|Age                |Numeric   |The Age of the player in years      |
|Total_duration     |Numeric(Double)  |The total amount of time that a player spent on the Mincraft server in seconds|
|Gender |Character | The self reported gender of the player  |
|num_session        |Integer|The total number of sessions played by the player|

**Summary of the dataset used**

Number of Observation: 123 observations

The variables used in the analysis: Age, num_session

The response variable: num_session

The explanatory variable: Age

First we loaded all the necessary libraries that we need to run out project.

In [ ]:
### Run this cell before continuing. 
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

In this step, we are reading and loading the players.csv file into R. 

In [ ]:
players<-read_csv("players.csv")
players

Now we also read and load in the sessions.csv file into the R program.

In [ ]:
sessions<-read_csv("sessions.csv")
sessions

**In this step we are wrangling and cleaning the data**

What is done here is that we have used the mutate function to create a session_duration column in our dataframe, this is done by using the original_end_time and the original_start_time. Right after we use the filter function, so that we are able to obly includw rows that contain hashed email and session_duration.

Once that has occured, we now use the group_by function to group all the hashedEmail together and finally we use the summarize function to dtermine the number of session (we call it num_session) and the total_duration(by summing the session_duration). We call this new dataframe tidy_sessions. 

In [ ]:
tidy_sessions<-sessions|>
mutate(session_duration = original_end_time - original_start_time)|>
filter(!is.na(hashedEmail), !is.na(session_duration))|>
group_by(hashedEmail)|>
summarize(num_session = n(), total_duration = sum(session_duration, na.rm = TRUE))
tidy_sessions

In this step, we are now selecting the hashedEmail, gender and Age columns and then filter for rows that have Age them and no NA values. We use the semi_join function to assure that rows with hashedEmail that are present in the tidy_sessions dataframe are kept. We name the object tidy_players.  

In [ ]:
tidy_players<-players|>
select(hashedEmail, gender, Age)|>
filter(!is.na(Age))|>
semi_join(tidy_sessions, by = "hashedEmail")
tidy_players

Next, we use inner_join on the tidy_sessions data and use that to merge the tidy_sessions and tidy_players into one dataframe. We name the object player_session. 

In [ ]:
player_session<-tidy_players|>
inner_join(tidy_sessions, by = "hashedEmail")
player_session

Now in this step, we are using the group_by function so that we group together the Age column and then summarize right after to calculate the mean of the num_session. We call this new column avg_session. We call the object of this step avg_player_session. 

In [ ]:
avg_player_session<-player_session|>
group_by(Age)|>
summarize(avg_session = mean(num_session))
avg_player_session

In this step, we are going to create a scatterplot so that we can plot the Age and avg_session. We will put Age on the x-axis and avg_session on the y-xis. This graph allows for us to the average sessions that are played by players based on their ages. 

In [ ]:
library(ggplot2)

options(repr.plot.width=12, repr.plot.height=7)
game_plot_1<-avg_player_session|>
ggplot(aes(x = Age, y = avg_session))+
geom_point(alpha = 1)+
labs(title = "Average Sessions Played Based on Age", x = "Age of Players", y = "Average Sessions Played")+
theme_minimal()
game_plot_1

Now, we are going to move on to the analysis portion of the project. For the data analysis, we have decided to use the linear regression model. The reason we have decided to use this method is because we have a continuous response and explanatory variable. These are numerical variables, so first of all it makes sense for us to use regression. Linear regression allows for us to interpret the data and easily predict how the number of sessions could possibly change with age. 

Before we started linear regression, we have to assume that the relationship between age and number of sessions would be linear (later, we see that is not really the case). We also need to consider the weaknesses of linear regression. We need to understand that when we plot and if the relationship is not linear, linear regression will not demonstrate it that well. It also crucial to consider any outliers in the data, which could affect the line of best fit to be altered. 

The reason I chose linear regression was because it seemed like a straightforward model to determine the relationship between age and number of sessions. 

Now, we are going to move on to the analysis portion of the project. We are first going to use the initial_split function, so that we can spit our player_session dataset into a training and testing set. In the initial_split function, we have decided to use 85% of the data and are splitting based on the num_session. We name the training set game_training and the testing set as game_testing. 

In [ ]:
game_split<-initial_split(player_session, prop = 0.85, strata = num_session)
game_training<-training(game_split)
game_testing<-testing(game_split)
game_split

Right after, we are going to set up a linear regression specification model. We use the linear_reg function to do this and we set_engine as "lm" to indicate we are use the linear regression engine and then we use set_mode to set as "regression", so that we know that we doing a regression problem. We name our object lm_spec. 

In [ ]:
lm_spec<-linear_reg()|>
set_engine("lm")|>
set_mode("regression")
lm_spec

Now, we are going to create a recipe and workflow, so that we can preprocess and combine the data together. We first use the recipe function and set our response variable as num_sessions and our predictor variable as Age. We also specify that we are using the game_training set. We name the object for the recipe, lm_recipe. 

Then, we create a workflow to combine our recipe and model together. At the end of the workflow, we use the fit function to fit our data on the game_training dataset. We name the workflow object as lm_fit. 

In [ ]:
lm_recipe<-recipe(num_session ~ Age, data = game_training)

lm_fit<-workflow()|>
add_recipe(lm_recipe)|>
add_model(lm_spec)|>
fit(data = game_training)
lm_fit

In this step of our analysis, we are going to make predictions and add those predictions to our game_training dataset. This is done by using our lm_fit data and using the predict funtion on the game_training dataset. Then we use the bind_cols function to attach our predictions on to game_training dataset. We name the object game_preds. 

Right after, we are going to create a scatter plot. We will set Age on our x-axis and num_session on our y-axis. From there, we also use the geom_line function to show our linear line here. We make the x variable of the line as Age and the y-variable as .pred. Then we decide to color the line blue. We then give descriptive axis titles and a title for the graph. We name the plot object as lm_predictions_plot. This plot is used to assess the model's fit and to see if it is a good representation of the data. (We use coord_cartesian to zoom in on most relevant range of the ages and the number of sessions). 

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
game_preds<-lm_fit|>
predict(game_training)|>
bind_cols(game_training)

lm_predictions_plot<-game_preds|>
ggplot(aes(x = Age, y = num_session))+
geom_point(alpha = 0.4)+
coord_cartesian(xlim = c(10, 50), ylim = c(0, 100)) +
geom_line(mapping = aes(x = Age, y = .pred), 
            color = "blue") +
        xlab("Age of Players") +
        ylab("The Number of Sessions played by Players") +
        theme(text = element_text(size = 20))
lm_predictions_plot

Now we are going to see how well our linear regression model works on our testing set. We use the predict function here on the game_testing data and apply our lm_fit to make our predictions. We then use the bind_cols to attach our predictions to our game_testing dataframe. Right after that, we use the metrics function to compute the RMSE. We set our truth variable as num_session, which is our actual data and our estimate as .pred, which is where we made our predictions. We name our object here lm_test_results. 

Then, we are going to use the filter function to do the .metric=="rmse", and the select function to get the rmse from the .estimate coumn and then the pull function to pull out the RMSE value that we need. 

In [ ]:
lm_test_results <- lm_fit |>
                    predict(game_testing) |>
                    bind_cols(game_testing) |>
                    metrics(truth = num_session, estimate = .pred)                    

lm_rmspe <- lm_test_results |>
                filter(.metric == 'rmse') |>
                select(.estimate) |>
                pull()
lm_rmspe

Finally, we are going to create a visualization of linear regression model on our test data. We create predictions again using our game_testing data and using the lm_fit data that created. We then use the bind_cols to add our prediction to the game_testing dataframe. We name the object test_preds. We do this step again because this time, we don't need to predict the RMSE, just the predictions. 

Now we create a plot to see how well our linear regression model works on data that has not been seen yet. We create a scatterplot where we set Age as the x-axis and the num_session on the y-axis. We then use the geom_line function to add in our best-fit line which is created by the x-variable (Age) and y-variable (num_session) and we color the line a blue colour. We then set descriptive labels and a title for our graph. We name our plot, lm_predictions_test_plot (Y-axis shown on log scale to better visualize session frequency distribution.) (geom_jitter is also used because the points were overlapping way too much). 

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 10)


test_preds <- lm_fit |>
    predict(game_testing) |>
    bind_cols(game_testing)

lm_predictions_test_plot <- test_preds |>
    ggplot(aes(x = Age, y = num_session)) +
        geom_point(alpha = 0.4) +
geom_jitter(alpha = 0.4, width = 0.3, height = 0.3) +
        geom_line(
            mapping = aes(x = Age, y = .pred), 
            color = "blue") +
  scale_y_log10() +
        xlab("Age of Players") +
        ylab("Number of Sessions Of Players") +
        theme(text = element_text(size = 20))
lm_predictions_test_plot

**Discussion**
From the plots and the data analysis that we have done, we were able to calculate the RMSE from linear regression, as well as see how our linear regression model acts on the training set of data and unseen data (testing set). When create the recipe and fit our model in the workflow, we get our equation of the line. Our equation of line is num_sessions = 18.6936 - 0.2928 * Age. The intercept (18.6936) is the predicted number of sessions for a player with an age = 0. The slope (-0.2928) indicates to us that in each additional year of age, the predicted number of sessions decreases by a value of around 0.29. This indicates to us that older indivdiuals tend to have less sessions compared to younger individuals. This equation indicates to use that age and number of sessions have a slight negative relationship. This could suggest to us that younger players tend to have more sessions and be more active on the minecraft server. 

My number of sessions seem to range from 0 to 100 sessions. My calculated RMSE value was 28.7. This indicates how off my number of sessions are from the actual number of sessions that occur. This seems to be a pretty big error and this could indicate that only age might not be a strong predictor for how many sessions a player tends to play. This could indicate that the relationship between age and num_session might just be very week, which is why we are seeing a large RMSE. This might indicate thato other variables other than age might play a bigger role in determining the number of sessions. This could also explain why when we look at the plots, our line of best-fit does not seem to go through the data points. The model is trying to find something to minimize error, however it leads to a trend not being followed. 

This outcome was a little expected because it makes sense that not only age wil affect the number of sessions by players. There can be many other variables like lifestyle, personality, etc that could have a greater effect on the number of sessions. We did see a weak relationship, so it could have some effect, but we can see that it does not have a large effect. Now we know that the relationship between age and number of sessions is fairly weak, this opens up space for questions that can explore other variables, such as can someone's past experiences in a game lead to a greater number of sessions? Or, we can also test gender to see if that would have an effect on the number of sessions one partakes in. The impact of these findings is that now that we know age is not a strong predictor for number of sessions, we know that using promotional techniques to promote the Minecraft server to target age groups would not be that effective due to the data we have seen today. This indicates to game developers that we need to use non-age based promotional techniques instead. 